# Testing Bayesian Optimisation for Function Optimisation

### TODO
- rather than evaluating the objective at the same places, choose xs randomly

In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
import time
import numpy as np
import GPy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import ipywidgets as widgets
from IPython.display import display
from IPython.core.debugger import set_trace

In [ ]:
import function_bo as fbo
from function_bo_plotting import *

In [ ]:
xmin, xmax = 0, 10
ymin, ymax = -5, 5

def to_fit(x):
    return np.sin(x) * 2*np.cos(x/4)

def to_fit_noisy(x):
    size = x.size if isinstance(x, np.ndarray) else None
    return to_fit(x) + np.random.normal(loc=0, scale=0.2, size=size)

def plot_to_fit():
    xs = np.linspace(xmin, xmax, num=100)
    plt.plot(xs, [to_fit(x) for x in xs], 'k--', label='to fit')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()
plot_to_fit()

**by specifying a different kernel or providing a mean function, many different types of functions can be sampled**

In [ ]:
def plot_random_sample():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 6))
    
    domain_bounds = ('x', xmin, xmax)
    range_bounds = (ymin, ymax)
    op = fbo.Optimiser(None, domain_bounds, range_bounds, 1)
    
    empty_trial = lambda f: fbo.Optimiser.Trial(f, [], None, None)
    
    np.random.seed(0)
    f = op.select_random()
    plot_trial_area(op, empty_trial(f), true_best=to_fit, quiet=True, ax=ax1)
    ax1.set_title('default (RBF)')
    
    np.random.seed(0)
    mu = lambda x: 0.2*x + 0.3*np.sin(x)
    f = op.select_random(mu=mu)
    plot_trial_area(op, empty_trial(f), true_best=to_fit, quiet=True, ax=ax2)
    xs = np.linspace(xmin, xmax, num=100)
    ax2.plot(xs, mu(xs), label=r'$\mu(x)$')
    ax2.legend()
    ax2.set_title('default with mean function')
    
    f = op.select_random(GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=0.1))
    plot_trial_area(op, empty_trial(f), true_best=to_fit, quiet=True, ax=ax3)
    ax3.set_title('Matern with small length scale')
plot_random_sample()

In [ ]:
def objective(f):
    # global reward
    R_g = integrate(lambda x: (f(x) - to_fit(x))**2, (xmin, xmax))
    # local rewards
    R_ls = []
    for x in np.linspace(xmin, xmax, num=10):
        R_l = (f(x)-to_fit(x))**2
        R_ls.append((x, R_l))
    return R_ls, R_g

domain_bounds = ('x', xmin, xmax)
range_bounds = (ymin, ymax)
op = fbo.Optimiser(objective, domain_bounds, range_bounds, desired_extremum='min', pre_phase_trials=2)

In [ ]:
np.random.seed(0)
op.run(10)

In [ ]:
plot_convergence(op, best_R_g=0)
plot_trials(op, op.trials, to_fit, color_by_reward=True)

In [ ]:
inc_i, inc = op.get_incumbent()
print('incumbent = trial {}'.format(inc_i))
plot_trial_area(op, inc, to_fit)

In [ ]:
plot_surrogate_with_trials(op, -1, to_fit, midpoint_fraction=0.1)

In [ ]:
t = op.trials[-1]
plot_acquisition(op, t.surrogate, np.array([0]), np.array([[-1, 1]]), l=2)

In [ ]:
plot_surrogate_3D(op, op.trials[-1].surrogate, flip_z=True)